In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import warnings
import random
import os

# --- Basic Setup ---
warnings.filterwarnings('ignore')

def set_seed(seed_value=42):
    """Sets the seed for reproducibility."""
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# --- Hyperparameters ---
INPUT_STEPS = 48
FORECAST_HOUR = 20
HIDDEN_SIZE = 64
NUM_LAYERS = 2
LEARNING_RATE = 0.001
BATCH_SIZE = 64
NUM_EPOCHS = 200
EARLY_STOPPING_PATIENCE = 10

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- Data Loading ---
def load_and_merge_data():
    """Loads all data files, merges them, and sets a datetime index."""
    try:
        old_chatia_df = pd.read_excel('Chatia_train.xlsx')
        old_rewaghat_df = pd.read_excel('Rewaghat_train.xlsx')
        new_chatia_df = pd.read_excel('Chatia_test.xlsx')
        new_rewaghat_df = pd.read_excel('Rewaghat_test.xlsx')
        rainfall_df = pd.read_excel('rainfall_data.xlsx')
        dumariaghat_df = pd.read_excel('Dumariaghat_data.xlsx')
    except FileNotFoundError as e:
        print(f"Error: {e}. Please ensure all data files are in the script's directory.")
        exit()

    full_chatia_df = pd.concat([old_chatia_df, new_chatia_df]).drop_duplicates(subset=['Date'])
    full_rewaghat_df = pd.concat([old_rewaghat_df, new_rewaghat_df]).drop_duplicates(subset=['Date'])

    for df_ in [full_chatia_df, full_rewaghat_df, rainfall_df, dumariaghat_df]:
        df_['Date'] = pd.to_datetime(df_['Date'], format='%d-%m-%Y %H:%M')

    dumariaghat_df = dumariaghat_df.drop_duplicates(subset=['Date'])

    df_merged = pd.merge(full_chatia_df, full_rewaghat_df, on='Date', how='inner')
    df_merged = pd.merge(df_merged, dumariaghat_df[['Date', 'Dumariaghat']], on='Date', how='inner')
    df_merged = pd.merge(df_merged, rainfall_df, on='Date', how='inner')

    df_merged.set_index('Date', inplace=True)
    df_merged.sort_index(inplace=True)
    df_merged.rename(columns={'Chatia': 'Chatia', 'Rewaghat': 'Rewaghat', 'Dumariaghat': 'Dumariaghat', 'Rainfall': 'Rainfall'}, inplace=True)
    
    print("All datasets loaded and merged successfully.")
    print(f"Full data spans from {df_merged.index.min()} to {df_merged.index.max()}")
    return df_merged[['Chatia', 'Rewaghat', 'Dumariaghat', 'Rainfall']]

# --- LSTM Data Preparation ---
def create_sequences_for_single_step(data, input_steps, forecast_hour):
    """Creates input sequences (X) and a single target value (y) for the LSTM."""
    X, y = [], []
    for i in range(len(data) - input_steps - forecast_hour + 1):
        X.append(data[i:(i + input_steps)])
        # Target 'y' is the single value at the 20th hour ahead (Rewaghat is column index 1)
        y.append(data[i + input_steps + forecast_hour - 1, 1])
    return np.array(X), np.array(y).reshape(-1, 1)

# --- LSTM Model Definition ---
class LSTMModel(nn.Module):
    """Sequence-to-Vector LSTM model that predicts a single future time step."""
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, 1) # Output size is 1 for single-step prediction

    def forward(self, x):
        h0 = torch.zeros(NUM_LAYERS, x.size(0), HIDDEN_SIZE).to(x.device)
        c0 = torch.zeros(NUM_LAYERS, x.size(0), HIDDEN_SIZE).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :]) # Use only the last time step's output
        return out

# --- Training Function ---
def train_network(model, train_loader, val_loader, criterion, optimizer):
    """Trains the LSTM model with early stopping."""
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(NUM_EPOCHS):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
                outputs = model(X_batch)
                val_loss += criterion(outputs, y_batch).item()
        val_loss /= len(val_loader)

        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Validation Loss: {val_loss:.6f}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
            torch.save(best_model_state, 'best_lstm_comparison_model.pth')
        else:
            patience_counter += 1

        if patience_counter >= EARLY_STOPPING_PATIENCE:
            print("Early stopping triggered.")
            break
            
    if best_model_state:
        model.load_state_dict(best_model_state)
    return model

# --- Evaluation Function (Mirrors XGBoost output) ---
def evaluate_seasonal_accuracy(model, test_df, test_dates, scaler):
    """Evaluates the model and prints accuracy for different seasons."""
    model.eval()
    
    # 1. Prepare test data and make predictions
    scaled_test_data = scaler.transform(test_df)
    X_test_seq, y_test_seq = create_sequences_for_single_step(scaled_test_data, INPUT_STEPS, FORECAST_HOUR)
    test_dataset = TensorDataset(torch.FloatTensor(X_test_seq), torch.FloatTensor(y_test_seq))
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    predictions = []
    with torch.no_grad():
        for X_batch, _ in test_loader:
            X_batch = X_batch.to(DEVICE)
            outputs = model(X_batch)
            predictions.extend(outputs.cpu().numpy())

    predictions = np.array(predictions).flatten()
    actuals = y_test_seq.flatten()

    # 2. Inverse transform predictions and actuals
    dummy_preds = np.zeros((len(predictions), 4))
    dummy_preds[:, 1] = predictions
    inv_predictions = scaler.inverse_transform(dummy_preds)[:, 1]

    dummy_actuals = np.zeros((len(actuals), 4))
    dummy_actuals[:, 1] = actuals
    inv_actuals = scaler.inverse_transform(dummy_actuals)[:, 1]

    # 3. Create results DataFrame with correct dates
    result_dates = test_dates[INPUT_STEPS + FORECAST_HOUR - 1:]
    results_df = pd.DataFrame({
        'Date': result_dates,
        'Actual': inv_actuals,
        'Predicted': inv_predictions
    }).set_index('Date')
    
    # 4. Filter by season (same as XGBoost notebook)
    monsoon_months = [6, 7, 8, 9, 10]
    monsoon_df = results_df[results_df.index.month.isin(monsoon_months)]
    dry_df = results_df[~results_df.index.month.isin(monsoon_months)]
    
    def calculate_custom_accuracy(df):
        if df.empty: return 0.0
        errors = df['Predicted'] - df['Actual']
        return np.mean((errors >= -0.15) & (errors <= 0.15)) * 100
        
    # 5. Calculate accuracies
    accuracy_monsoon = calculate_custom_accuracy(monsoon_df)
    accuracy_dry = calculate_custom_accuracy(dry_df)
    accuracy_full = calculate_custom_accuracy(results_df)

    # 6. Print in the desired format
    print("\n--- LSTM Model Performance Summary (20-Hour Forecast) ---")
    print(f"1. Custom Accuracy for Monsoon Season (June-Oct): {accuracy_monsoon:.2f}%")
    print(f"2. Custom Accuracy for Full Year: {accuracy_full:.2f}%")
    print(f"3. Custom Accuracy for Dry Season (Nov-May): {accuracy_dry:.2f}%")

# --- Main Execution Block ---
if __name__ == "__main__":
    # Load and merge all data
    df = load_and_merge_data()
    
    # ** NEW: Time-based split identical to XGBoost notebook **
    train_df = df.loc[df.index < (df.index.max() - pd.DateOffset(years=2))]
    val_df = df.loc[(df.index >= (df.index.max() - pd.DateOffset(years=2))) & 
                    (df.index < (df.index.max() - pd.DateOffset(years=1)))]
    test_df = df.loc[df.index >= (df.index.max() - pd.DateOffset(years=1))]
    
    print(f"\nTrain set shape: {train_df.shape}")
    print(f"Validation set shape: {val_df.shape}")
    print(f"Test set shape: {test_df.shape}")

    # Scale the data
    scaler = MinMaxScaler()
    scaled_train_data = scaler.fit_transform(train_df)
    scaled_val_data = scaler.transform(val_df)
    
    # Create sequences for the LSTM
    X_train, y_train = create_sequences_for_single_step(scaled_train_data, INPUT_STEPS, FORECAST_HOUR)
    X_val, y_val = create_sequences_for_single_step(scaled_val_data, INPUT_STEPS, FORECAST_HOUR)

    # Create DataLoaders
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Initialize model, loss function, and optimizer
    model = LSTMModel(input_size=4, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS).to(DEVICE)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Train the model
    print("\nStarting LSTM model training for 20-hour forecast...")
    trained_model = train_network(model, train_loader, val_loader, criterion, optimizer)
    
    # Evaluate and print results in the desired format
    evaluate_seasonal_accuracy(trained_model, test_df, test_df.index, scaler)

Using device: cuda
All datasets loaded and merged successfully.
Full data spans from 2014-03-01 00:00:00 to 2024-10-31 23:00:00

Train set shape: (76007, 4)
Validation set shape: (8760, 4)
Test set shape: (8777, 4)

Starting LSTM model training for 20-hour forecast...
Epoch [1/200], Validation Loss: 0.001111
Epoch [2/200], Validation Loss: 0.000974
Epoch [3/200], Validation Loss: 0.000924
Epoch [4/200], Validation Loss: 0.000807
Epoch [5/200], Validation Loss: 0.000770
Epoch [6/200], Validation Loss: 0.000762
Epoch [7/200], Validation Loss: 0.000843
Epoch [8/200], Validation Loss: 0.000741
Epoch [9/200], Validation Loss: 0.000709
Epoch [10/200], Validation Loss: 0.000760
Epoch [11/200], Validation Loss: 0.000686
Epoch [12/200], Validation Loss: 0.000763
Epoch [13/200], Validation Loss: 0.000692
Epoch [14/200], Validation Loss: 0.000716
Epoch [15/200], Validation Loss: 0.000687
Epoch [16/200], Validation Loss: 0.000761
Epoch [17/200], Validation Loss: 0.000687
Epoch [18/200], Validation